# Домашнее задание к лекции "Базовые понятия статистики"
Будем осуществлять работу с непростым набором данных ( https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv ) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

## Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

___
## Решение задания 1

In [337]:
import pandas as pd
import numpy as np

In [406]:
# Читаем файл
df = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv')

# Создаем копию датафрейма, оставляя в нем только 8 столбцов
df_comp = df.iloc[:, [0, 1, 3, 4, 5, 7, 10, 22]].copy()

# Переименовываем столбцы
df_comp.columns = ['Surgery', 'Age', 'Rectal_Temp', 'Pulse', 'Respiratory_Rate', 'Temp_Of_Extrem', 'Pain', 'Outcome' ]

# Для столбцов с неполными данными заменяем ? на NaN
for clmn in ('Surgery', 'Age', 'Rectal_Temp', 'Pulse', 'Respiratory_Rate', 'Temp_Of_Extrem', 'Pain', 'Outcome'):
    df_comp.loc[df_comp[clmn] == '?', clmn] = np.nan

df_comp.head(300)


,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
0,1,1.0,39.2,88,20,NaN,3,3
1,2,1.0,38.30,40,24,1,3,1
2,1,9.0,39.10,164,84,1,2,2
3,2,1.0,37.30,104,35,NaN,NaN,2
4,2,1.0,NaN,NaN,NaN,1,2,1
...,...,...,...,...,...,...,...,...
294,1,1.0,NaN,120,70,NaN,2,3
295,2,1.0,37.20,72,24,2,4,3
296,1,1.0,37.50,72,30,3,4,2
297,1,1.0,36.50,100,24,3,3,1


## Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

___
## Решение задания 2

In [407]:
"""
В таблице оставлены следующие столбцы, имеющие соответствующие ограничения
(описание отсюда: https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names)

Surgery:          1 - Хирургическое вмешательство, 2 - Без хирургии
Age:              1 - взрослая лошадь,  2 - молодая (меньше полугода)
Rectal_Temp:      Ректальная температура в градусах. Норма - 37.8. Высокая - при инфекциях, пониженная - глубокий шок
Pulse:            Удары в минуту. 30-40 - норма, ниже - у спортивных. Повышенный - болезненные повреждения, шок
Respiratory_Rate: Частота дыхания. Норма 8-10, показатель параметра сомнителен из-за высоких флуктуаций
Temp_Of_Extrem:   Внутренняя температура. 1 - норма, 2 - тепло, 3 - прохладно, 4 - холодно. 
                  3, 4 - шок, 2 - корреляция с повышенной ректальной температурой
Pain:             Субъективная оценка уровня боли. 1 - тревога, 2 - депрессия, 3 - временами слабая боль, 
                  4 - временами сильная боль, 5 - постоянная сильная боль                    
Outcome:          Результат лечения: 1 - выжила, 2 - умерла, 3 - была усыплена
"""
# Смотрим типы данных в колонках и количество пропусков
df_comp.info()
df_comp.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Surgery           298 non-null    object 
 1   Age               299 non-null    float64
 2   Rectal_Temp       239 non-null    object 
 3   Pulse             275 non-null    object 
 4   Respiratory_Rate  241 non-null    object 
 5   Temp_Of_Extrem    230 non-null    object 
 6   Pain              244 non-null    object 
 7   Outcome           298 non-null    object 
dtypes: float64(1), object(7)
memory usage: 18.8+ KB


,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
count,298,299.00000,239,275,241,230,244,298
unique,2,NaN,65,52,40,4,5,3
top,1,NaN,38.00,48,20,1,3,1
freq,180,NaN,16,28,28,115,67,178
mean,NaN,1.64214,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,2.17730,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN


In [419]:
# По автоматически подсчитанным данным видно, что только один из столбцов определен как числовой (возраст)
# Преобразуем типы столбцов и посмотрим, что изменилось
df_comp = df_comp.apply(pd.to_numeric)
# Автоматическое преобразование к оптимальному типу
df_comp = df_comp.convert_dtypes()
df_comp.head(20)

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
0,1,1,39.2,88,20,<NA>,3,3
1,2,1,38.3,40,24,1,3,1
2,1,9,39.1,164,84,1,2,2
3,2,1,37.3,104,35,<NA>,<NA>,2
4,2,1,<NA>,<NA>,<NA>,1,2,1
5,1,1,37.9,48,16,1,3,1
6,1,1,<NA>,60,<NA>,<NA>,<NA>,2
7,2,1,<NA>,80,36,4,4,3
8,2,9,38.3,90,<NA>,<NA>,5,1
9,1,1,38.1,66,12,3,3,1


In [402]:
# Смотрим, что изменилось с автоматически вычисляемыми метриками выборки:
df_comp.info()
df_comp.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Surgery           298 non-null    Int64  
 1   Age               299 non-null    Int64  
 2   Rectal_Temp       239 non-null    Float64
 3   Pulse             275 non-null    Int64  
 4   Respiratory_Rate  241 non-null    Int64  
 5   Temp_Of_Extrem    230 non-null    Int64  
 6   Pain              244 non-null    Int64  
 7   Outcome           298 non-null    Int64  
dtypes: Float64(1), Int64(7)
memory usage: 21.1 KB


,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
count,298.000000,299.00000,239.000000,275.000000,241.000000,230.000000,244.000000,298.000000
mean,1.395973,1.64214,38.166527,71.934545,30.427386,2.013043,2.942623,1.550336
std,0.489881,2.17730,0.733508,28.680522,17.678256,1.042672,1.303993,0.737967
min,1.000000,1.00000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.200000,64.000000,24.000000,1.500000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.00000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


In [420]:
# Разбираемся более подробно со столбцами
# (В датафрейме 299 строк)

# Surgery - по определению может принимать одно из двух значений - 1 или 2. 
# Тип данных - Качественные, номинальные
# По автоматически подсчитанным данным для этой колонки видно, что в одной строке данные отсутствуют,
# Подсчет среднего, std тут кажется бессмысленным.
df_comp.Surgery.value_counts()
#df_comp[df_comp['Surgery'].isnull()]


,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
131,<NA>,1,38.0,48,20,1,4,<NA>


In [343]:
# Age - может принимать одно из двух значений - 1 (взрослая) или 2 (молодая) 
# Тип данных - Качественные, номинальные
# По автоматически подсчитанным данным для этой колонки видно, что пропусков нет,
# Подсчет среднего, std тут также бессмыслен.
# Также в колонке присутствуют значения 9, выходящие за рамки допустимых (1 или 2)
# а значение 2 отсутствует вовсе. Можно предположить, что по ошибке вместо 2 везде вводили 9
# Другая гипотеза - все лошади взрослые и 9 вводили по ошибке вместо 1
df_comp.Age.value_counts()
#df_comp[df_comp['Age'].isnull()]

1    275
9     24
Name: Age, dtype: Int64

In [344]:
# Rectal_Temp - в текущем датасете меняется в диапазоне от 35.4 до 40.8 градуса Цельсия
# Тип данных - Количественные, непрерывные
# Для этой колонки довольно много пропусков (60)
# К выбросам можно было бы отнести температуру ниже 37 градусов и выше 40. Таких значений мало,
# но они сильно отличаются от большинства. Тем не менее выбрасывать их не стоит, они могут быть корректными
print('Пропуски', len(df_comp) - df_comp.Rectal_Temp.count())
print('При норме 37.8 градусов:')
print('Минимум', df_comp.Rectal_Temp.min())
print('Квантиль 5%', df_comp.Rectal_Temp.quantile(.05))
print('Квантиль 10%', df_comp.Rectal_Temp.quantile(.1))
print('Квантиль 25%', df_comp.Rectal_Temp.quantile(.25))
print('Квантиль 75%', df_comp.Rectal_Temp.quantile(.75))
print('Квантиль 90%', df_comp.Rectal_Temp.quantile(.9))
print('Квантиль 95%', df_comp.Rectal_Temp.quantile(.95))
print('Максимум', df_comp.Rectal_Temp.max())
print('Медиана', df_comp.Rectal_Temp.median())
print('Среднее', df_comp.Rectal_Temp.mean())
print('Мода', df_comp.Rectal_Temp.mode()[0])
print('Среднеквадратичное отклонение', df_comp.Rectal_Temp.std())
#df_comp.Rectal_Temp.value_counts()
#df_comp[df_comp['Rectal_Temp'].isnull()]
# Проверка расчетов и прочие персентили
df_comp.Rectal_Temp.describe()

Пропуски 60
При норме 37.8 градусов:
Минимум 35.4
Квантиль 5% 37.1
Квантиль 10% 37.3
Квантиль 25% 37.8
Квантиль 75% 38.5
Квантиль 90% 39.1
Квантиль 95% 39.4
Максимум 40.8
Медиана 38.2
Среднее 38.166527196652716
Мода 38.0
Среднеквадратичное отклонение 0.7335083691537396


count    239.000000
mean      38.166527
std        0.733508
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: Rectal_Temp, dtype: float64

In [137]:
# Pulse - в текущем датасете пульс меняется в диапазоне от 30 до 184 ударов в минуту
# Тип данных - Количественные, дискретные (на уровне данных округление до целого, но по сути - непрерывная величина)
# Для этой колонки пропусков меньше, чем для температуры (24)
# Формально выбросами можно считать значения выше 113 (квантиль 90%) или выше 125 (квантиль 95%)
# но было бы правильно проверить, нет ли корреляции у высоких значений с другими показателями.
# Тогда это может быть отражением реальной клинической картины, а не погрешностями измерения
print('Пропуски', len(df_comp) - df_comp.Pulse.count())
print('При норме 30 - 40 ударов в минуту:')
print('Минимум', df_comp.Pulse.min())
print('Квантиль 25%', df_comp.Pulse.quantile(.25))
print('Квантиль 75%', df_comp.Pulse.quantile(.75))
print('Квантиль 90%', df_comp.Pulse.quantile(.9))
print('Квантиль 95%', df_comp.Pulse.quantile(.95))
print('Максимум', df_comp.Pulse.max())
print('Медиана', df_comp.Pulse.median())
print('Среднее', df_comp.Pulse.mean())
print('Мода', df_comp.Pulse.mode()[0])
print('Среднеквадратичное отклонение', df_comp.Pulse.std())
#df_comp.Pulse.value_counts()
df_comp[df_comp['Pulse'].isnull()]
# Проверка расчетов и прочие персентили
#df_comp.Pulse.describe()

Пропуски 24
При норме 30 - 40 ударов в минуту:
Минимум 30
Квантиль 25% 48.0
Квантиль 75% 88.0
Квантиль 90% 113.19999999999999
Квантиль 95% 125.20000000000005
Максимум 184
Медиана 64.0
Среднее 71.93454545454546
Мода 48
Среднеквадратичное отклонение 28.680522003654737


,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
4,2,1,<NA>,<NA>,<NA>,1,2,1
27,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,2
51,2,1,<NA>,<NA>,<NA>,1,1,1
55,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,1
57,1,1,<NA>,<NA>,20,3,5,2
73,1,9,<NA>,<NA>,<NA>,<NA>,<NA>,2
77,1,1,<NA>,<NA>,<NA>,3,5,2
82,1,1,38.0,<NA>,24,3,5,2
92,2,1,<NA>,<NA>,<NA>,3,5,2
114,2,1,<NA>,<NA>,40,1,3,1


In [142]:
# Respiratory_Rate - в текущем датасете частота дыхания меняется в диапазоне от 8 до 96 ударов в минуту
# Тип данных - Количественные, дискретные (на уровне данных округление до целого, но по сути - непрерывная величина)
# Для этой колонки пропусков немало (58)
# Формально выбросами можно считать значения выше 50 (квантиль 90%) или выше 70 (квантиль 95%)
# но было бы правильно проверить, нет ли корреляции у высоких значений с другими показателями
print('Пропуски', len(df_comp) - df_comp.Respiratory_Rate.count())
print('При норме 8 - 10 циклов в минуту:')
print('Минимум', df_comp.Respiratory_Rate.min())
print('Квантиль 25%', df_comp.Respiratory_Rate.quantile(.25))
print('Квантиль 75%', df_comp.Respiratory_Rate.quantile(.75))
print('Квантиль 95%', df_comp.Respiratory_Rate.quantile(.95))
print('Максимум', df_comp.Respiratory_Rate.max())
print('Медиана', df_comp.Respiratory_Rate.median())
print('Мода', df_comp.Respiratory_Rate.mode()[0])
print('Среднее', df_comp.Respiratory_Rate.mean())
print('Среднеквадратичное отклонение', df_comp.Respiratory_Rate.std())
#df_comp.Respiratory_Rate.value_counts()
#df_comp[df_comp['Respiratory_Rate'].isnull()]
# Проверка расчетов и прочие персентили
df_comp.Respiratory_Rate.describe()
# Проверка гипотезы о том, что чем выше пульс, тем вероятнее смерть лошади (гипотеза не подтвердилась)
#df_comp.sort_values(by=['Respiratory_Rate'], ascending=False)[['Respiratory_Rate', 'Outcome']].head(50)

Пропуски 58
При норме 8 - 10 циклов в минуту:
Минимум 8
Квантиль 25% 18.0
Квантиль 75% 36.0
Квантиль 95% 70.0
Максимум 96
Медиана 24.0
Мода 20
Среднее 30.42738589211618
Среднеквадратичное отклонение 17.678256330531212


count    241.000000
mean      30.427386
std       17.678256
min        8.000000
25%       18.000000
50%       24.000000
75%       36.000000
max       96.000000
Name: Respiratory_Rate, dtype: float64

In [129]:
# Temp_Of_Extrem - в текущем датасете внутренняя температура определяется по четырехуровневой шкале
# Тип данных - Качественные, номинальные ()
# Для этой колонки пропусков немало (69)
# Выбросов или некорректных значений (кроме пропусков) не выявлено
print('Пропуски', len(df_comp) - df_comp.Temp_Of_Extrem.count())
print('Норма - значение 1')
print('Мода', df_comp.Temp_Of_Extrem.mode()[0])
print('Для такого показателя наиболее наглядно частотное распределение значений:')
df_comp.Temp_Of_Extrem.value_counts()
#df_comp[df_comp['Temp_Of_Extrem'].isnull()]
# Проверка расчетов и прочие персентили
#df_comp.Temp_Of_Extrem.describe()
# Проверка гипотезы о том, что чем выше пульс, тем вероятнее смерть лошади (гипотеза не подтвердилась)
#df_comp.sort_values(by=['Respiratory_Rate'], ascending=False)[['Respiratory_Rate', 'Outcome']].head(50)

Пропуски 69
Норма - значение 1
Мода 1
Для такого показателя наиболее показательно частотное распределение значений:


1    115
3    102
4      8
2      5
Name: Temp_Of_Extrem, dtype: Int64

In [131]:
# Pain - в текущем датасете уровень боли определяется по пятиуровневой шкале
# Тип данных - Качественные, порядковые. Боль возрастает по шкале от 1 (тревога) до 5 (постоянная сильная боль)
# Для этой колонки пропусков немало (55)
# Выбросов или некорректных значений (кроме пропусков) не выявлено, данные распределились более-менее равномерно,
# поэтому и расчет основных показателей никаких особенностей не показывает
print('Пропуски', len(df_comp) - df_comp.Pain.count())
print('Минимум', df_comp.Pain.min())
print('Квантиль 25%', df_comp.Pain.quantile(.25))
print('Квантиль 75%', df_comp.Pain.quantile(.75))
print('Максимум', df_comp.Pain.max())
print('Медиана', df_comp.Pain.median())
print('Мода', df_comp.Pain.mode()[0])
print('Среднее', df_comp.Pain.mean())
print('Среднеквадратичное отклонение', df_comp.Pain.std())
print('Для такого показателя наиболее наглядно частотное распределение значений:')
df_comp.Pain.value_counts()
#df_comp[df_comp['Pain'].isnull()]
# Проверка расчетов и прочие персентили
#df_comp.Pain.describe()

Пропуски 55
Минимум 1
Квантиль 25% 2.0
Квантиль 75% 4.0
Максимум 5
Медиана 3.0
Мода 3
Среднее 2.942622950819672
Среднеквадратичное отклонение 1.303993109678259
Для такого показателя наиболее наглядно частотное распределение значений:


3    67
2    59
5    41
4    39
1    38
Name: Pain, dtype: Int64

In [132]:
# Outcome - в текущем датасете результат лечения определяется как один из трех возможных исходов (выжила, умерла, усыплена)
# Тип данных - Качественные, номинальные.  
# Для этой колонки пропусков практически нет (всего 1)
# Выбросов или некорректных значений (кроме пропуска) не выявлено, значительную часть лошадей удалось вылечить (что приятно)
# Расчет основных показателей здесь бессмысленен, имеет смысл лишь статистическое распределение
print('Пропуски', len(df_comp) - df_comp.Outcome.count())
print('Мода', df_comp.Outcome.mode()[0])
df_comp.Outcome.value_counts()
#df_comp[df_comp['Outcome'].isnull()]
# Проверка расчетов и прочие персентили
#df_comp.Outcome.describe()

Пропуски 1
Мода 1


1    178
2     76
3     44
Name: Outcome, dtype: Int64

## Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

___
## Решение задания 3

In [183]:
# В результате анализа датафрейма выявлены следующие пропуски по столбцам:
# Surgery 1
# Age 0
# Rectal_Temp 60
# Pulse 25
# Respiratory_Rate 58
# Temp_Of_Extrem 69
# Pain 55
# Outcome 1
# При этом оказалось, что колонка Age неинформативна. Поскольку ее дополнять не надо, то ее мы будем считать стартовой.



In [403]:
# В колонке Surgery всего один пропуск.Смотрим данные по этой строке
df_comp[df_comp['Surgery'].isnull()]

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
131,<NA>,1,38.0,48,20,1,4,<NA>


In [287]:
# Surgery
# Кажется рациональным посмотреть, каков процент лошадей с таким высоким уровнем боли подвергался хирургическому вмешательству
# Значения в других колонках кажутся не предельными.
#df_comp[ (df_comp['Pain'] == 5) ]['Surgery'].value_counts()
#df_comp[ (df_comp['Pain'] == 5) ]['Surgery'].mode()[0]
#Более правильно проанализировать эту корреляцию и для других уровней боли. Похоже, она есть:
df_comp.groupby('Pain')['Surgery'].value_counts()

Pain  Surgery
1     2          29
      1           9
2     1          34
      2          25
3     1          39
      2          28
4     1          28
      2          10
5     1          33
      2           8
Name: Surgery, dtype: int64

In [423]:
#Фиксим столбец Surgery:
# Фактически нам надо вычислить моду для того уровня боли, который мы берем для анализа

def define_surgery(cur_pain):
    return df_comp[ (df_comp['Pain'] == cur_pain) ]['Surgery'].mode()[0]
df_comp.loc[(pd.isnull(df_comp.Surgery)), 'Surgery'] = define_surgery(df_comp.Pain)

df_comp[df_comp['Surgery'].isnull()]
#df_comp['Surgery'].count()

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome


In [347]:
# В колонке Outcome всего один пропуск.Смотрим данные по этой строке
df_comp[df_comp['Outcome'].isnull()]

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
131,1,1,38.0,48,20,1,4,<NA>


In [348]:
# Outcome
# Проверяем гипотезу, что хирургическое вмешательство как-то влияет на результат:
# df_comp.groupby('Surgery')['Outcome'].value_counts()
# Проверяем гипотезу, что сила боли + хирургия как-то влияет на результат:
df_comp.groupby(['Pain', 'Surgery'])['Outcome'].value_counts()
# Видим, что конкретно для случая с единственным пропуском в нашем случае можно предположить, 
# что эта лошадь умерла с вероятностью около 75 процентов.

Pain  Surgery  Outcome
1     1        1           9
      2        1          26
               2           2
               3           1
2     1        1          15
               2          11
               3           8
      2        1          18
               3           4
               2           3
3     1        1          33
               2           5
               3           1
      2        1          21
               3           4
               2           3
4     1        2          15
               1           8
               3           5
      2        3           6
               1           2
               2           2
5     1        2          17
               1          10
               3           6
      2        2           6
               1           2
Name: Outcome, dtype: int64

In [349]:
#Фиксим последний столбец Outcome:
def define_outcome(cur_surgery, cur_pain):
    return df_comp[ (df_comp['Surgery'] == cur_surgery) & (df_comp['Pain'] == cur_pain) ]['Outcome'].mode()[0]

df_comp.loc[(pd.isnull(df_comp.Outcome)), 'Outcome'] = define_outcome(df_comp.Surgery, df_comp.Pain)

df_comp[df_comp['Outcome'].isnull()]
#df_comp['Surgery'].count()

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome


In [350]:
# В колонке Pulse относительно немного пропусков
df_comp[df_comp['Pulse'].isnull()]

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
4,2,1,<NA>,<NA>,<NA>,1,2,1
27,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,2
51,2,1,<NA>,<NA>,<NA>,1,1,1
55,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,1
57,1,1,<NA>,<NA>,20,3,5,2
73,1,9,<NA>,<NA>,<NA>,<NA>,<NA>,2
77,1,1,<NA>,<NA>,<NA>,3,5,2
82,1,1,38.0,<NA>,24,3,5,2
92,2,1,<NA>,<NA>,<NA>,3,5,2
114,2,1,<NA>,<NA>,40,1,3,1


In [351]:
# На данный момент у нас не осталось пропусков в колонках Surgery и Outcome. 
# Выдвигаем гипотезу, что среднее значение пульса для лошадей с одинаковыми исходом и хирургией можно рассматривать как критерий
df_comp.groupby(['Surgery', 'Outcome'])['Pulse'].mean()
# Видно, что для выживших лошадей, как с операцией так и без, средний пульс всегда заметно ниже, так что корреляция есть.

Surgery  Outcome
1        1          68.483146
         2          88.183673
         3          80.037037
2        1          57.448718
         2             85.625
         3             84.625
Name: Pulse, dtype: Float64

In [352]:
#Фиксим столбец Pulse:
def define_pulse(cur_surgery, cur_outcome):
    return round(df_comp[ (df_comp['Surgery'] == cur_surgery) & (df_comp['Outcome'] == cur_outcome) ]['Pulse'].mean())

df_comp.loc[(pd.isnull(df_comp.Pulse)), 'Pulse'] = define_pulse(df_comp.Surgery, df_comp.Outcome)

df_comp[df_comp['Pulse'].isnull()]
#df_comp['Surgery'].count()

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome


In [353]:
# В колонке Respiratory_Rate, судя по описанию датасета, возможны флуктуации, данные здесь неточные.
df_comp[df_comp['Respiratory_Rate'].isnull()]

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
4,2,1,<NA>,72,<NA>,1,2,1
6,1,1,<NA>,60,<NA>,<NA>,<NA>,2
8,2,9,38.3,90,<NA>,<NA>,5,1
27,1,1,<NA>,72,<NA>,<NA>,<NA>,2
28,2,1,37.7,48,<NA>,1,1,1
31,1,1,37.2,60,<NA>,1,3,1
36,1,1,37.8,72,<NA>,3,5,1
37,2,1,38.6,52,<NA>,1,3,1
39,1,1,<NA>,88,<NA>,3,5,2
42,1,1,<NA>,120,<NA>,4,4,3


In [354]:
# Проверим гипотезу о том, есть ли корреляция частоты дыхания с пульсом и исходом лечения. 
df_comp.groupby(['Outcome', 'Pulse'])['Respiratory_Rate'].mean().tail(50)
# Также проверяем наличие связи частоты пульса на частоту дыхания (независимо от исхода) 
#df_comp.groupby('Pulse')['Respiratory_Rate'].mean().head(50)
# Обе проверки показывают, что есть слабая тенденция увеличения частоты дыхания при повышении температуры

Outcome  Pulse
2        66       20.0
         68       17.0
         70       16.0
         72       28.0
         75       36.0
         78       <NA>
         80       44.0
         84       38.5
         86       20.0
         88       30.5
         90       30.0
         92       28.0
         96       35.5
         98       35.0
         100      <NA>
         104      35.0
         114      36.0
         120      24.0
         124      36.0
         128      36.0
         136      48.0
         146      96.0
         150      50.0
         160      20.0
         164      84.0
         184      84.0
3        36       16.0
         40       24.0
         48       30.0
         56       68.0
         60       23.2
         65       24.0
         66       20.0
         70       22.0
         72       24.0
         76       <NA>
         78       24.0
         80       38.0
         82       12.0
         84       30.0
         88       47.5
         90       40.0
         100      5

In [355]:
# Фиксим столбец Respiratory_Rate:
# Вычисляем соотношение между средней частотой пульса и средней частотой дыхания
pulse_to_respiratory = df_comp['Pulse'].mean() / df_comp['Respiratory_Rate'].mean()
df_comp.Respiratory_Rate = df_comp.Respiratory_Rate.fillna(df_comp.Pulse // pulse_to_respiratory)
df_comp[df_comp['Respiratory_Rate'].isnull()]
#df_comp['Respiratory_Rate'].count()

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome


In [356]:
# В колонке Rectal_Temp отображается температура
df_comp[df_comp['Rectal_Temp'].isnull()]

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
4,2,1,<NA>,72,30,1,2,1
6,1,1,<NA>,60,25,<NA>,<NA>,2
7,2,1,<NA>,80,36,4,4,3
15,1,9,<NA>,128,36,3,4,2
27,1,1,<NA>,72,30,<NA>,<NA>,2
33,1,1,<NA>,100,30,3,5,1
34,2,1,<NA>,104,24,3,4,3
39,1,1,<NA>,88,37,3,5,2
42,1,1,<NA>,120,50,4,4,3
44,2,1,<NA>,120,50,3,5,2


In [357]:
# Проверим гипотезу о том, есть ли корреляция температуры с пульсом и исходом лечения. 
df_comp.groupby(['Outcome', 'Pulse'])['Rectal_Temp'].mean().head(60)

Outcome  Pulse
1        30            38.5
         36            37.2
         38            37.6
         40       38.078571
         42       38.071429
         44       38.044444
         45            38.1
         46            38.6
         48       37.973077
         49            38.4
         50       37.866667
         52       38.271429
         54       38.414286
         56           38.14
         60       38.185714
         64       38.733333
         66            38.1
         68       38.133333
         70            <NA>
         72            38.6
         76            38.3
         78            36.5
         80           38.05
         84            39.5
         88           38.25
         90            38.3
         92           38.45
         96            38.0
         100           37.3
         104          38.45
         108           38.8
         110           39.4
         112           <NA>
         120      38.633333
         124           38.2
     

In [358]:
# Также проверяем наличие связи с Temp_of_Extrem
df_comp.groupby(['Outcome', 'Temp_Of_Extrem'])['Rectal_Temp'].mean().head(50)


Outcome  Temp_Of_Extrem
1        1                 38.152326
         2                 38.233333
         3                 38.188235
         4                      <NA>
2        1                 38.161538
         3                 38.353571
         4                      <NA>
3        1                 37.916667
         2                     37.65
         3                 38.093333
         4                     39.25
Name: Rectal_Temp, dtype: Float64

In [359]:
# А что с пульсом?
df_comp.groupby('Pulse')['Rectal_Temp'].mean().head(50)

Pulse
30         38.15
36          37.6
38          37.6
40        38.025
42         37.74
44         38.04
45          38.1
46          38.6
48     37.942857
49          38.4
50     37.866667
52         38.19
54     38.414286
56     38.066667
60        38.045
64     38.657143
65          <NA>
66         37.86
68        38.075
70          38.1
72     38.541176
75          37.1
76          38.3
78         38.25
80         38.44
82          37.8
84     37.914286
86          38.5
88         38.04
90     37.866667
92          38.6
96     38.083333
98          36.4
100    38.433333
104    38.066667
108    38.266667
110         39.4
112         38.3
114         40.3
120         38.7
124         38.5
128         <NA>
129         38.5
130        38.25
132         38.3
136         38.1
140         36.7
146         39.2
150         38.9
160         38.6
Name: Rectal_Temp, dtype: Float64

In [360]:
# Ни одна из трех гипотез не подтвердилась. В этой связи есть большой соблазн просто поставить среднее значение..
# Фиксим столбец Rectal_Temp:
df_comp.Rectal_Temp = df_comp.Rectal_Temp.fillna(df_comp.Rectal_Temp.mean())
df_comp[df_comp['Rectal_Temp'].isnull()]
#df_comp['Respiratory_Rate'].count()

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome


In [361]:
# В колонке Pain - субъективная оценка уровня боли по пятибальной шкале. Смотрим пропуски:
df_comp[df_comp['Pain'].isnull()]

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
3,2,1,37.3,104,35,<NA>,<NA>,2
6,1,1,38.166527,60,25,<NA>,<NA>,2
16,2,1,37.5,48,24,<NA>,<NA>,1
18,2,1,39.4,110,35,3,<NA>,1
23,1,1,38.1,60,12,3,<NA>,1
24,2,1,37.8,60,42,<NA>,<NA>,1
26,1,1,37.8,48,12,1,<NA>,1
27,1,1,38.166527,72,30,<NA>,<NA>,2
35,2,1,38.3,112,16,3,<NA>,3
38,1,9,39.2,146,96,<NA>,<NA>,2


In [362]:
# Проверяем наличие связи с Outcome и Surgery
df_comp.groupby(['Surgery', 'Outcome', 'Pain'])['Pain'].count()
# Видим, что для каждой комбинации Surgery и Outcome находятся максимумы по количеству лошадей с определенным уровнем боли

Surgery  Outcome  Pain
1        1        1        9
                  2       15
                  3       33
                  4        9
                  5       10
         2        2       11
                  3        5
                  4       15
                  5       17
         3        2        8
                  3        1
                  4        5
                  5        6
2        1        1       26
                  2       18
                  3       21
                  4        2
                  5        2
         2        1        2
                  2        3
                  3        3
                  4        2
                  5        6
         3        1        1
                  2        4
                  3        4
                  4        6
Name: Pain, dtype: int64

In [385]:
# Фактически я хочу найти моду для каждой группировки, а вычисление моды впрямую в transform не предполагается.
# Подсказка к ниже приведенной конструкции найдена здесь: 
# https://medium.com/analytics-vidhya/best-way-to-impute-categorical-data-using-groupby-mean-mode-2dc5f5d4e12d
df_comp['Pain'] = df_comp.groupby(['Surgery', 'Outcome'], sort=False)['Pain'].apply(lambda x: x.fillna(x.mode().iloc[0]))
#df_comp[df_tmp['Pain'].isnull()]
df_comp.head(20)

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
0,1,1,39.2,88,20,<NA>,3,3
1,2,1,38.3,40,24,1,3,1
2,1,9,39.1,164,84,1,2,2
3,2,1,37.3,104,35,<NA>,5,2
4,2,1,38.166527,72,30,1,2,1
5,1,1,37.9,48,16,1,3,1
6,1,1,38.166527,60,25,<NA>,5,2
7,2,1,38.166527,80,36,4,4,3
8,2,9,38.3,90,38,<NA>,5,1
9,1,1,38.1,66,12,3,3,1


In [386]:
# Последняя оставшаяся колонка Temp_Of_Extrem
df_comp[df_comp['Temp_Of_Extrem'].isnull()].head(50)

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
0,1,1,39.2,88,20,<NA>,3,3
3,2,1,37.3,104,35,<NA>,5,2
6,1,1,38.166527,60,25,<NA>,5,2
8,2,9,38.3,90,38,<NA>,5,1
10,2,1,39.1,72,52,<NA>,2,1
16,2,1,37.5,48,24,<NA>,1,1
20,2,1,38.4,48,16,<NA>,1,1
24,2,1,37.8,60,42,<NA>,1,1
27,1,1,38.166527,72,30,<NA>,5,2
38,1,9,39.2,146,96,<NA>,5,2


In [388]:
# Проверяем наличие связи с Outcome и Surgery
df_comp.groupby(['Surgery', 'Outcome', 'Temp_Of_Extrem'])['Temp_Of_Extrem'].count()
# Почти для каждой комбинации Surgery и Outcome находится явный максимум по кол-ву лошадей с определенным уровнем температуры

Surgery  Outcome  Temp_Of_Extrem
1        1        1                 41
                  3                 34
                  4                  1
         2        1                 11
                  3                 29
                  4                  2
         3        1                  3
                  2                  1
                  3                 10
                  4                  3
2        1        1                 53
                  2                  3
                  3                  8
         2        1                  3
                  3                 12
                  4                  1
         3        1                  4
                  2                  1
                  3                  9
                  4                  1
Name: Temp_Of_Extrem, dtype: int64

In [389]:
# Используем для заполнения пропусков тот же алгоритм, как и для уровня боли.
df_comp['Temp_Of_Extrem'] = df_comp.groupby(['Surgery', 'Outcome'], sort=False)['Temp_Of_Extrem'].apply(lambda x: x.fillna(x.mode().iloc[0]))
#df_comp[df_tmp['Pain'].isnull()]
df_comp.head(20)

,Surgery,Age,Rectal_Temp,Pulse,Respiratory_Rate,Temp_Of_Extrem,Pain,Outcome
0,1,1,39.2,88,20,3,3,3
1,2,1,38.3,40,24,1,3,1
2,1,9,39.1,164,84,1,2,2
3,2,1,37.3,104,35,3,5,2
4,2,1,38.166527,72,30,1,2,1
5,1,1,37.9,48,16,1,3,1
6,1,1,38.166527,60,25,3,5,2
7,2,1,38.166527,80,36,4,4,3
8,2,9,38.3,90,38,1,5,1
9,1,1,38.1,66,12,3,3,1


In [398]:
# Есть смысл округлить дополненные значения Rectal_Temp с точностью до одного знака после запятой (проглядел)
df_comp['Rectal_Temp'] = df_comp['Rectal_Temp'].apply(lambda x: round(np.float64(x), 1))
# Также сохраняем полученный датафрейм в новом файле для дальнейшего возможного анализа
df_comp.to_csv('horse_data_updated.csv')
df_tmp.head(20)